In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from torchmetrics.functional import accuracy
from common_utils import Accumulator


def topIndices(vec, x, largest):
    sorted_idx = torch.argsort(vec, descending=largest)
    top_x_idxs, other_idxs = (sorted_idx[:x], sorted_idx[x:])
    return top_x_idxs, other_idxs

def irreducibleLoss(data=None, target=None, global_index=None, 
                              irreducible_loss_model=None, target_device=None):
    
    if type(irreducible_loss_model) is torch.Tensor:
        if (target_device is not None and irreducible_loss_model.device != target_device ):
            irreducible_loss_model = irreducible_loss_model.to(device=target_device)
        irreducible_loss = irreducible_loss_model[global_index]
    else:
        irreducible_loss = F.cross_entropy(irreducible_loss_model(data), target, reduction="none")
    return irreducible_loss

class ReducibleLossSelection:
    bald = False

    def __call__(self, selected_batch_size, data=None, target=None, global_index=None, large_model=None,
        irreducible_loss_model=None):

        with torch.no_grad():

            model_loss = F.cross_entropy( large_model(data), target, reduction="none")

            irreducible_loss = irreducibleLoss(data, target, global_index, irreducible_loss_model, model_loss.device)

            reducible_loss = model_loss - irreducible_loss

            top_x_idxs, _ = topIndices( reducible_loss, selected_batch_size, largest=True )
            selected_irreducible_loss = irreducible_loss[top_x_idxs]

        return top_x_idxs, selected_irreducible_loss
    
selection_method = ReducibleLossSelection()
update_irreducible = True

def train_irr_loss(
        irreducible_loss_clf, 
        data_loader,
        loss_fn, 
        opt, 
        accumulator : Accumulator, 
        selection__mode = True, presample = 3.0 ):
    
    # BEGIN Solution (do not delete this comment!)
    ret_loss, correct = (0.0, 0)
    irreducible_loss_clf.train(True)  # Set model to training mode
    
    for x, labels in data_loader:
        x, labels = x.cuda(), labels.cuda()
        
        opt.zero_grad()
        all_pred = irreducible_loss_clf(x)
        loss = loss_fn(all_pred, labels)

        loss.backward()
        opt.step()
        
        ret_loss += loss.item()
        pred = all_pred.max(1)[1]
        correct += (pred==labels).sum().item()

    ret_loss = ret_loss / len(data_loader)
    accuracy = correct / len(data_loader.dataset)

    return ret_loss, accuracy


def train_batch_rho_loss(
        large_model,
        irreducible_loss_model,
        batch,
        loss_fn, 
        optimizer, 
        accumulator : Accumulator, 
        selection__mode = True, presample = 3.0 ):


    global_index, data, target = batch
    batch_size = len(data)
    selected_batch_size = max(1, int(batch_size /presample))


    large_model.eval() 

    selected_indices,  irreducible_loss = selection_method.__call__(
        selected_batch_size=selected_batch_size,
        data=data,
        target=target,
        global_index=global_index,
        large_model=large_model,
        irreducible_loss_model=irreducible_loss_model,
    ) 

    large_model.train()  # switch to eval mode to compute selection

    data, target = data[selected_indices], target[selected_indices]

    optimizer.zero_grad()
    logits = large_model(data)
    loss = loss_fn(logits, target) 

    mean_loss = loss.mean()
    mean_loss.backward()
    optimizer.step()

    # training metrics
    preds = torch.argmax(F.log_softmax(logits, dim=1), dim=1)
    acc = accuracy(preds, target)

    accumulator.average( 
        train_loss = ( mean_loss.mean(), 1) ,
        train_acc = ( acc, 1) )


In [ ]:
import copy

def train_full(model, train_dataloader, loss_fn, optimizer, n_epochs, eval = None, callback=None, presample=2, tau_th = None):
    irreducible_loss_model = copy.deepcopy(model)


    large_batch = int( train_dataloader.batch_size)
    
    if callback :
        callback.setMeta(
            large_batch = large_batch,
            n_epochs = n_epochs, 
            presample = presample, 
            tau_th = tau_th)

    d = model.device
    epochs = tqdm(range(n_epochs), desc='Epochs', leave=True)
    for i_epoch in epochs:
        accum = Accumulator()
        
        
        for idxs, X_batch, y_batch in train_dataloader:
            train_batch_rho_loss(model, 
                                irreducible_loss_model,
                                (idxs, X_batch.to(d), y_batch.to(d)),
                                loss_fn,
                                optimizer, 
                                accum,
                                presample)

        if callback :
            val_scores = eval(model) if eval else {}
            cb_dict = callback( **accum.getAll(), **val_scores)
            epochs.set_postfix(cb_dict)

In [ ]:
def evaluate(model, dataloader, loss_fn):
    model.eval()
    logits = []
    targets = []
    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            output = model(X_batch.to(model.device)).cpu()
            logits.append(output)
            targets.append(y_batch)
    logits = torch.cat(logits)
    targets = torch.cat(targets)
    loss = loss_fn(logits, targets).mean().item()
    acc = (logits.argmax(dim=1) == targets).sum().item() / len(targets)
    return loss, acc


In [ ]:
from data_loaders import train_dataloader, test_dataloader

train_loader  = train_dataloader(batch_size=120)#128*3
test_loader = test_dataloader(batch_size=120)

In [ ]:
%load_ext autoreload

%aimport common_utils
%autoreload 1

In [ ]:
%autoreload 1
from models import ResNet50
from common_utils import UnCallBack

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = ResNet50()
model.to(device)
model.device = device

loss_fn = nn.CrossEntropyLoss(reduction='none')
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3 )

callback = UnCallBack( info_list = ['train_loss', 'train_acc', 'train_w_loss', 'val_loss', 'val_acc', 'train_uniform_cnt'])

def eval_callback(model):
    loss, acc =evaluate(model, test_loader, loss_fn)
    return {"val_loss": loss, "val_acc": acc}

train_full(model, 
           train_loader, 
           loss_fn, 
           optimizer, 
           n_epochs=40, 
           eval=eval_callback, 
           callback=callback, 
           presample=3, 
           tau_th = None)


callback.save("rho_loss")